In [1]:
import socketio
import numpy as np

def lua():
    sio = socketio.Client()
    sio.connect('http://nothanks.sonjh02.me:3001')
    sio.emit('take_seat', f'lua{i}')
    
    @sio.event
    def your_turn(data):
        hand = data['you']['hand']
        coin = data['you']['coin']
        card = data['open']
        pot = data['coin']
        gap = np.abs(np.r_[hand] - card)
        if gap.size > 0:
            gap = gap.min()
        else:
            gap = 33
        if gap <= 1 or card - pot <= 3:
            return False
        if coin > 10:
            return True
        if np.random.randint(1, card) < pot + 11 - coin:
            return False
        return True
    
def me():
    sio = socketio.Client()
    sio.connect('http://nothanks.sonjh02.me:3001')
    sio.emit('take_seat', f'손재현{i}')
    
    def gap(h, c):
        g = 33
        for v in h:
            g = np.clip(np.abs(v-c), 0, g)
        return g
    
    @sio.event
    def your_turn(data):
        opponents = data['opponents']
        hand = data['you']['hand']
        coin = data['you']['coin']
        card = data['open']
        pot = data['coin']
        
        if gap(hand, card) == 1:
            contender = False
            for opponent in opponents:
                if gap(opponent['hand'], card) == 1:
                    contender = True
                    return False
            return np.random.randint(1, card) > pot + 15 + len(opponents) - coin
        else:
            ch = False
            cl = False
            for opponent in opponents:
                if gap(opponent['hand'], card+1) == 1:
                    ch = True
                if gap(opponent['hand'], card-1) == 1:
                    cl = True
                    
            if ch or cl:
                return np.random.randint(1, card) > pot + 3 - coin
            else:
                if len(hand) == 0:
                    return np.random.randint(1, card) > pot + 10 - coin 
                else:
                    return np.random.randint(1, card) > pot + 5 - coin


In [4]:
i = 1
me()

In [5]:
i = 2
me()

In [6]:
i = 3
me()